# Week 6: Output Parser 進階 RAG 應用

## 課程目標
- 掌握模型量化技術（4-bit quantization）
- 學習 LangChain Output Parser 各種類型
- 實作結構化 RAG 輸出系統
- 應用於實際商業場景

## 教學大綱
1. 環境設定與量化技術
2. Output Parser 基礎
3. RAG + Parser 整合
4. 商業應用案例

## Part 1: 環境設定

In [1]:
# Cell 1: 安裝必要套件
!pip install -q langchain-huggingface langchain-community langchain
!pip install -q transformers accelerate bitsandbytes
!pip install -q pypdf faiss-cpu==1.10.0
!pip install -q pydantic
!pip install -q rank-bm25  # BM25 檢索
!pip install -q sentence-transformers  # Reranker

print("✅ 套件安裝完成")
print("⚠️  請重啟 Runtime 後再執行後續 cells")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 10.2 MB/s eta 0:00:00
✅ 套件安裝完成
⚠️  請重啟 Runtime 後再執行後續 cells


In [1]:
# Cell 2: HuggingFace 認證
from google.colab import userdata
from huggingface_hub import login

token = userdata.get('HF_TOKEN')
login(token=token)

print("✅ HuggingFace 認證成功")

✅ HuggingFace 認證成功


In [2]:
# Cell 3: 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 切換到資料目錄
%cd drive/MyDrive/data_rag

print("✅ Google Drive 掛載成功")

Mounted at /content/drive
/content/drive/MyDrive/data_rag
✅ Google Drive 掛載成功


In [1]:
# Cell 4: 導入基礎套件
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import (
    StrOutputParser,
    JsonOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser
)
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import json

print("✅ 套件導入完成")
print(f"🔧 PyTorch 版本: {torch.__version__}")
print(f"🎮 CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")

✅ 套件導入完成
🔧 PyTorch 版本: 2.8.0+cu126
🎮 CUDA 可用: True
📊 GPU: Tesla T4


## Part 2: 量化技術實戰

In [4]:
# Cell 5: 設定 4-bit 量化配置
print("⚙️  設定 4-bit 量化配置...")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 啟用 4-bit 量化
    bnb_4bit_compute_dtype=torch.bfloat16,  # 使用 bfloat16 計算
    bnb_4bit_quant_type="nf4",             # NormalFloat 4-bit 量化類型
    bnb_4bit_use_double_quant=True,        # 雙重量化，進一步壓縮
)

print("""\n📘 量化參數說明：
- load_in_4bit: 將模型權重量化為 4-bit（原本 16-bit）
- bnb_4bit_compute_dtype: 計算時使用的資料型別
- bnb_4bit_quant_type: NF4 (NormalFloat) 適合神經網路
- bnb_4bit_use_double_quant: 量化常數也進行量化

💡 預期效果：
- 記憶體使用減少約 75%
- 1B 模型從 ~2GB 降至 ~0.5GB
- 推理速度略有提升
- 精度損失極小（<1%）
""")

⚙️  設定 4-bit 量化配置...

📘 量化參數說明：
- load_in_4bit: 將模型權重量化為 4-bit（原本 16-bit）
- bnb_4bit_compute_dtype: 計算時使用的資料型別
- bnb_4bit_quant_type: NF4 (NormalFloat) 適合神經網路
- bnb_4bit_use_double_quant: 量化常數也進行量化

💡 預期效果：
- 記憶體使用減少約 75%
- 1B 模型從 ~2GB 降至 ~0.5GB
- 推理速度略有提升
- 精度損失極小（<1%）



In [ ]:
# Cell 6: 載入量化模型
print("🚀 載入量化模型 gemma-3-1b-it...\n")

llm = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    model_kwargs={"quantization_config": quantization_config},
    pipeline_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.9,
    }
)

chat_model = ChatHuggingFace(llm=llm)

print("\n✅ 模型載入成功！")
print("\n💾 記憶體使用情況：")
if torch.cuda.is_available():
    print(f"已分配: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"已保留: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

🚀 載入量化模型 gemma-3-1b-it...



tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

In [ ]:
# 安裝 OpenAI 套件
!pip install -q openai langchain-openai
print("✅ OpenAI 套件安裝完成")

### 設定 OpenAI API Key
請確保您已將 OpenAI API Key 儲存在 Colab 的 Secrets 中，名稱為 `OPENAI_API_KEY`。

In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.1 MB/s eta 0:00:00


In [8]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
import os

# 讀取 API Key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 建立 ChatOpenAI 實例
# 使用 gpt-4o 模型 (目前 OpenAI 最新的模型)
# 您也可以換成其他模型，例如 "gpt-3.5-turbogpt-5"
chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

print(f"✅ OpenAI Chat Model ({openai_chat_model.model_name}) 建立成功")

✅ OpenAI Chat Model (gpt-4o) 建立成功


現在 Cell 9 將使用 `openai_chat_model` 來代替之前的 `chat_model`。

In [9]:
# Cell 7: 測試量化模型
from langchain_core.messages import HumanMessage, SystemMessage
import os

# 設定 CUDA_LAUNCH_BLOCKING=1 以幫助除錯
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


print("🧪 測試量化模型輸出...\n")

messages = [
    SystemMessage("你是一個專業的 AI 助手，請用繁體中文簡潔回答。"),
    HumanMessage("什麼是模型量化？請用 2-3 句話說明。")
]

response = chat_model.invoke(messages)
print("🤖 模型回答：")
print(response.content)
print("\n✅ 量化模型測試成功！")

🧪 測試量化模型輸出...

🤖 模型回答：
模型量化是一種技術，用於減少機器學習模型的大小和計算需求，通常透過將浮點數權重轉換為較低精度的整數表示。這不僅能降低存儲和計算成本，還能加速推理過程，特別是在資源受限的設備上。

✅ 量化模型測試成功！


## Part 3: Output Parser 基礎教學

In [10]:
# Cell 8: StrOutputParser - 基本字串解析
print("📝 範例 1: StrOutputParser (字串解析器)\n")

str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="請用一句話解釋：{concept}",
    input_variables=["concept"]
)

chain = prompt | chat_model | str_parser

result = chain.invoke({"concept": "RAG (Retrieval-Augmented Generation)"})
print(f"輸出類型: {type(result)}")
print(f"內容: {result}")

📝 範例 1: StrOutputParser (字串解析器)

輸出類型: <class 'str'>
內容: RAG（檢索增強生成）是一種結合信息檢索和生成模型的方法，通過檢索相關資料來輔助生成更準確和上下文相關的文本。


In [11]:
# Cell 9: CommaSeparatedListOutputParser - 列表解析
print("📋 範例 2: CommaSeparatedListOutputParser (列表解析器)\n")

list_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="列出 5 個{category}，用逗號分隔，只輸出項目名稱：",
    input_variables=["category"]
)

# 將 chain 中的模型替換為 openai_chat_model
chain = prompt | chat_model | list_parser

result = chain.invoke({"category": "機器學習演算法"})
print(f"輸出類型: {type(result)}")
print(f"列表項目:")
for i, item in enumerate(result, 1):
    print(f"  {i}. {item.strip()}")

📋 範例 2: CommaSeparatedListOutputParser (列表解析器)

輸出類型: <class 'list'>
列表項目:
  1. 線性迴歸
  2. 決策樹
  3. 支持向量機
  4. K-均值聚類
  5. 隨機森林


In [13]:
chain = prompt | chat_model | str_parser
result = chain.invoke({"category": "機器學習演算法"})
result

'線性回歸, 決策樹, 支持向量機, K-均值聚類, 隨機森林'

In [14]:
type(result)

str

In [22]:
# Cell 10: JsonOutputParser - JSON 格式解析（改進版）
print("🔧 範例 3: JsonOutputParser (JSON 解析器)\n")
import langchain
langchain.debug = True
# 定義簡單的 JSON schema
class Product(BaseModel):
    name: str = Field(description="產品名稱")
    category: str = Field(description="產品類別")
    price: int = Field(description="價格（新台幣）")
    in_stock: bool = Field(description="是否有庫存")

json_parser = JsonOutputParser(pydantic_object=Product)

# 改進的 Prompt - 使用 Few-Shot 範例
prompt = PromptTemplate(
    template="""從產品描述中提取資訊並輸出 JSON 格式。

範例：
產品描述: iPhone 17 是 Apple 的智慧手機，售價 52900 元，目前缺貨
JSON: {{"name": "iPhone 17", "category": "智慧手機", "price": 52900, "in_stock": false}}

現在請處理：
產品描述: {description}
JSON:""",
    input_variables=["description"]
)

# # 建立 LLM（直接用參數，不使用 GenerationConfig 物件）
# json_llm = HuggingFacePipeline.from_model_id(
#     model_id="google/gemma-3-1b-it",
#     task="text-generation",
#     model_kwargs={
#         "quantization_config": quantization_config,
#         "device_map": "auto"
#     },
#     pipeline_kwargs={
#         "max_new_tokens": 128,
#         "do_sample": False,        # 不使用採樣（確定性輸出）
#         "pad_token_id": 0,         # 設定 padding token
#         "eos_token_id": 1          # 設定 end-of-sequence token
#     }
# )
# json_chat = ChatHuggingFace(llm=json_llm)


chain = prompt | chat_model | json_parser

description = "MacBook Pro 是 Apple 的專業筆記型電腦，售價 68900 元，目前有現貨"

print(f"📝 產品描述: {description}\n")



try:
    result = chain.invoke({"description": description})

    print(f"✅ 解析成功！")
    print(f"輸出類型: {type(result)}\n")
    print(f"JSON 內容:")
    print(json.dumps(result, indent=2, ensure_ascii=False))

    # 驗證資料完整性
    print(f"\n📦 資料驗證:")
    required_fields = ["name", "category", "price", "in_stock"]
    missing_fields = [f for f in required_fields if f not in result]

    if missing_fields:
        print(f"  ⚠️  缺少欄位: {', '.join(missing_fields)}")
    else:
        print(f"  ✅ 所有必要欄位都存在")
        print(f"\n📋 產品資訊:")
        print(f"  名稱: {result['name']}")
        print(f"  類別: {result['category']}")
        print(f"  價格: NT$ {result['price']:,}")
        print(f"  庫存: {'有貨 ✅' if result['in_stock'] else '缺貨 ❌'}")

except Exception as e:
    print(f"❌ 解析錯誤: {e}")

    # 除錯：顯示原始 LLM 輸出
    print(f"\n🔍 除錯資訊 - LLM 原始輸出:")
    try:
        debug_chain = prompt | json_chat
        debug_result = debug_chain.invoke({"description": description})
        print(debug_result.content[:500])  # 只顯示前 500 字元

        # 嘗試手動解析
        print(f"\n💡 嘗試手動解析 JSON...")
        import re
        json_match = re.search(r'\{[^{}]+\}', debug_result.content)
        if json_match:
            manual_json = json.loads(json_match.group())
            print(f"✅ 手動解析成功:")
            print(json.dumps(manual_json, indent=2, ensure_ascii=False))
        else:
            print("❌ 找不到 JSON 格式")

    except Exception as debug_e:
        print(f"除錯失敗: {debug_e}")

    print("\n💡 調整建議：")
    print("- 已使用直接參數設定確定性輸出")
    print("- 如果持續失敗，建議改用 PydanticOutputParser (見 Cell 11)")
    print("- 或使用更簡化的 prompt 和更小的模型輸出")

🔧 範例 3: JsonOutputParser (JSON 解析器)

📝 產品描述: MacBook Pro 是 Apple 的專業筆記型電腦，售價 68900 元，目前有現貨

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "description": "MacBook Pro 是 Apple 的專業筆記型電腦，售價 68900 元，目前有現貨"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "description": "MacBook Pro 是 Apple 的專業筆記型電腦，售價 68900 元，目前有現貨"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 從產品描述中提取資訊並輸出 JSON 格式。\n\n範例：\n產品描述: iPhone 17 是 Apple 的智慧手機，售價 52900 元，目前缺貨\nJSON: {\"name\": \"iPhone 17\", \"category\": \"智慧手機\", \"price\": 52900, \"in_stock\": false}\n\n現在請處理：\n產品描述: MacBook Pro 是 Apple 的專業筆記型電腦，售價 68900 元，目前有現貨\nJSON:"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.31s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text

In [23]:
# Cell 11: PydanticOutputParser - 型別安全的結構化輸出
print("🎯 範例 4: PydanticOutputParser (Pydantic 解析器)\n")

# 定義簡化的資料模型（減少欄位以提高成功率）
class TechArticleSummary(BaseModel):
    title: str = Field(description="文章標題")
    main_topic: str = Field(description="主要主題")
    key_points: List[str] = Field(description="關鍵重點，2-3 點")
    difficulty: str = Field(description="難度：初級/中級/進階")

pydantic_parser = PydanticOutputParser(pydantic_object=TechArticleSummary)

# 改進 Prompt - 使用 Few-Shot 範例，不使用 format_instructions
prompt = PromptTemplate(
    template="""分析技術文章並提取結構化資訊，以 JSON 格式輸出。

範例：
文章：Docker 是一個容器化平台，可以打包應用程式及其依賴。它簡化了部署流程，適合微服務架構。開發者可以確保環境一致性。
JSON: {{
  "title": "Docker 容器化技術介紹",
  "main_topic": "容器化與部署",
  "key_points": ["打包應用程式及依賴", "簡化部署流程", "確保環境一致性"],
  "difficulty": "中級"
}}

現在請分析：
文章：{article}
JSON:""",
    input_variables=["article"]
)

# # 建立穩定的 LLM（不使用 GenerationConfig 物件，直接用參數）
# stable_llm = HuggingFacePipeline.from_model_id(
#     model_id="google/gemma-3-1b-it",
#     task="text-generation",
#     model_kwargs={
#         "quantization_config": quantization_config,
#         "device_map": "auto"
#     },
#     pipeline_kwargs={
#         "max_new_tokens": 256,
#         "do_sample": False,        # 確定性輸出
#         "pad_token_id": 0,
#         "eos_token_id": 1
#     }
# )
# stable_chat = ChatHuggingFace(llm=stable_llm)

chain = prompt | chat_model | pydantic_parser

article = """RAG (Retrieval-Augmented Generation) 是一種結合檢索與生成的 AI 技術。
它先從知識庫中檢索相關資訊，再將檢索結果與用戶問題一起傳給大型語言模型生成答案。
這種方法可以有效減少 AI 幻覺問題，提供更準確且有依據的回答。"""

print(f"📝 文章內容:\n{article}\n")

try:
    result = chain.invoke({"article": article})
    print(f"✅ 解析成功！類型: {type(result)}\n")
    print(f"📌 標題: {result.title}")
    print(f"🎯 主題: {result.main_topic}")
    print(f"📊 難度: {result.difficulty}")
    print(f"\n💡 關鍵重點:")
    for i, point in enumerate(result.key_points, 1):
        print(f"  {i}. {point}")

except Exception as e:
    print(f"❌ 解析錯誤: {str(e)[:200]}...\n")

    # 除錯：顯示原始輸出
    print("🔍 除錯資訊 - LLM 原始輸出:")
    try:
        debug_chain = prompt | stable_chat
        debug_result = debug_chain.invoke({"article": article})
        print(debug_result.content[:500])

        # 嘗試手動解析
        print("\n💡 嘗試手動解析...")
        import re
        json_match = re.search(r'\{[^{}]*\[[^\]]*\][^{}]*\}|\{[^{}]+\}', debug_result.content, re.DOTALL)
        if json_match:
            try:
                manual_json = json.loads(json_match.group())
                print("✅ 找到 JSON 結構:")
                print(json.dumps(manual_json, indent=2, ensure_ascii=False))

                # 嘗試用找到的 JSON 建立物件
                manual_result = TechArticleSummary(**manual_json)
                print("\n✅ 手動建立 Pydantic 物件成功！")
                print(f"標題: {manual_result.title}")
                print(f"主題: {manual_result.main_topic}")
                print(f"難度: {manual_result.difficulty}")
                print(f"重點數量: {len(manual_result.key_points)}")

            except Exception as parse_e:
                print(f"❌ 手動解析也失敗: {parse_e}")
        else:
            print("❌ 找不到完整的 JSON 結構")

    except Exception as debug_e:
        print(f"除錯失敗: {debug_e}")

    print("\n💡 調整建議：")
    print("- 已簡化模型欄位（移除 target_audience）")
    print("- 已使用 Few-Shot 範例引導")
    print("- 已設定 do_sample=False 確保穩定性")
    print("- 可以嘗試使用更大的模型（gemma-3-4b-it）")
    print("- 或改用更簡單的 JsonOutputParser（見 Cell 10）")

🎯 範例 4: PydanticOutputParser (Pydantic 解析器)

📝 文章內容:
RAG (Retrieval-Augmented Generation) 是一種結合檢索與生成的 AI 技術。
它先從知識庫中檢索相關資訊，再將檢索結果與用戶問題一起傳給大型語言模型生成答案。
這種方法可以有效減少 AI 幻覺問題，提供更準確且有依據的回答。

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "article": "RAG (Retrieval-Augmented Generation) 是一種結合檢索與生成的 AI 技術。\n它先從知識庫中檢索相關資訊，再將檢索結果與用戶問題一起傳給大型語言模型生成答案。\n這種方法可以有效減少 AI 幻覺問題，提供更準確且有依據的回答。"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "article": "RAG (Retrieval-Augmented Generation) 是一種結合檢索與生成的 AI 技術。\n它先從知識庫中檢索相關資訊，再將檢索結果與用戶問題一起傳給大型語言模型生成答案。\n這種方法可以有效減少 AI 幻覺問題，提供更準確且有依據的回答。"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 分析技術文章並提取結構化資訊，以 JSON 格式輸出。\n\n範例：\n文章：Docker 是一個容器化平台，可以打包應用程式及其依賴。它簡化了部署流程，適合微服務架構。開發者可以確保環境一致性。\nJSON: {\n  \"title\": \"Docker 

In [ ]:
# Cell 12: Output Parser 比較總結
print("""📊 Output Parser 類型比較

┌─────────────────────────┬──────────────────┬─────────────────────┬─────────────┐
│ Parser 類型             │ 輸出類型         │ 適用場景            │ 難度        │
├─────────────────────────┼──────────────────┼─────────────────────┼─────────────┤
│ StrOutputParser         │ str              │ 簡單文字回答        │ ⭐          │
│ CommaSeparatedList      │ List[str]        │ 列表項目            │ ⭐⭐        │
│ JsonOutputParser        │ Dict             │ 結構化資料          │ ⭐⭐⭐      │
│ PydanticOutputParser    │ Pydantic Model   │ 型別安全資料        │ ⭐⭐⭐⭐    │
└─────────────────────────┴──────────────────┴─────────────────────┴─────────────┘

💡 選擇建議：
1. 簡單回答 → StrOutputParser
2. 需要列表 → CommaSeparatedListOutputParser
3. 需要驗證 → PydanticOutputParser (推薦！)
4. 彈性資料 → JsonOutputParser

⚠️  注意事項：
- 結構化輸出建議使用 temperature=0 或極低值
- Pydantic 提供自動型別檢查和驗證
- 複雜結構可能需要多次重試
""")

## Part 4: RAG 系統建立

In [ ]:
# Cell 13: 載入論文資料
print("📚 載入學術論文...\n")

# 使用 Multi-Agent Debate 論文
pdf_path = "2509.05396v1.pdf"

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ 成功載入論文")
print(f"📄 總頁數: {len(documents)}")
print(f"📝 第一頁前 200 字元:\n{documents[0].page_content[:200]}...")

In [ ]:
# Cell 14: 文件分割與向量化
print("✂️  文件分割與向量化...\n")

# 文件分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

texts = text_splitter.split_documents(documents)
print(f"📦 分割成 {len(texts)} 個文字區塊")

# 建立 Embeddings
print("\n🔄 建立向量嵌入...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 建立向量資料庫
print("🗄️  建立 FAISS 向量資料庫...")
vectorstore = FAISS.from_documents(texts, embeddings)

print("\n✅ RAG 資料準備完成！")

In [ ]:
# Cell 15: 建立基礎 RAG Chain (無 Parser)
print("🔗 建立基礎 RAG Chain\n")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

rag_prompt = PromptTemplate(
    template="""根據以下資訊回答問題。如果資訊中沒有相關內容，請說你不知道。

相關資訊:
{context}

問題: {question}

回答:""",
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt},
    return_source_documents=True
)

# 測試基礎 RAG
query = "請問 multi-agent debate 的主要發現是什麼？"
result = qa_chain.invoke({"query": query})

print(f"❓ 問題: {query}\n")
print(f"💬 回答: {result['result']}\n")
print(f"📌 使用了 {len(result['source_documents'])} 個來源片段")

## Part 5: RAG + Output Parser 整合應用

In [29]:
# Cell 16: 案例 1 - 論文關鍵資訊提取
print("📄 案例 1: 學術論文關鍵資訊提取\n")

class PaperAnalysis(BaseModel):
    title: str = Field(description="論文標題")
    main_finding: str = Field(description="主要研究發現，1-2 句話")
    methodology: str = Field(description="研究方法，簡要說明")
    key_contributions: List[str] = Field(description="主要貢獻，2-3 點")
    confidence: float = Field(description="回答信心度 0-1", ge=0, le=1)

# 建立 Parser
paper_parser = PydanticOutputParser(pydantic_object=PaperAnalysis)
print(paper_parser.get_format_instructions())
# 建立結構化 RAG Prompt
paper_prompt = PromptTemplate(
    template="""根據以下論文片段，提取結構化資訊。

論文片段:
{context}

問題: {question}

{format_instructions}

分析結果:""",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": paper_parser.get_format_instructions()}
)

# Cell 3: 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 切換到資料目錄
%cd drive/MyDrive/data_rag

print("✅ Google Drive 掛載成功")

# Cell 13: 載入論文資料
print("📚 載入學術論文...\n")

# 使用 Multi-Agent Debate 論文
pdf_path = "2509.05396v1.pdf"

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ 成功載入論文")
print(f"📄 總頁數: {len(documents)}")
print(f"📝 第一頁前 200 字元:\n{documents[0].page_content[:200]}...")

# 手動實作 RAG + Parser 流程
query = "分析這篇論文的核心內容"
# Cell 14: 文件分割與向量化
print("✂️  文件分割與向量化...\n")

# 文件分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

texts = text_splitter.split_documents(documents)
print(f"📦 分割成 {len(texts)} 個文字區塊")

# 建立 Embeddings
print("\n🔄 建立向量嵌入...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 建立向量資料庫
print("🗄️  建立 FAISS 向量資料庫...")
vectorstore = FAISS.from_documents(texts, embeddings)

print("\n✅ RAG 資料準備完成！")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 1. 檢索相關文件
relevant_docs = retriever.get_relevant_documents(query)
context = "\n\n".join([doc.page_content for doc in relevant_docs[:3]])

# 2. 生成結構化 prompt
formatted_prompt = paper_prompt.format(context=context, question=query)

# 3. 呼叫 LLM
response = chat_model.invoke([HumanMessage(content=formatted_prompt)])

# 4. 解析輸出
try:
    analysis = paper_parser.parse(response.content)

    print("✅ 論文分析結果:\n")
    print(f"📌 標題: {analysis.title}")
    print(f"\n🔍 主要發現:\n{analysis.main_finding}")
    print(f"\n🛠️  研究方法:\n{analysis.methodology}")
    print(f"\n💡 主要貢獻:")
    for i, contrib in enumerate(analysis.key_contributions, 1):
        print(f"  {i}. {contrib}")
    print(f"\n📊 信心度: {analysis.confidence:.2f}")

except Exception as e:
    print(f"❌ 解析失敗: {e}")
    print(f"\n原始輸出:\n{response.content}")

📄 案例 1: 學術論文關鍵資訊提取

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"description": "論文標題", "title": "Title", "type": "string"}, "main_finding": {"description": "主要研究發現，1-2 句話", "title": "Main Finding", "type": "string"}, "methodology": {"description": "研究方法，簡要說明", "title": "Methodology", "type": "string"}, "key_contributions": {"description": "主要貢獻，2-3 點", "items": {"type": "string"}, "title": "Key Contributions", "type": "array"}, "confidence": {"description": "回答信心度 0-1", "maximum": 1, "minimum": 0, "title": "Confidence", "type": "number"}}, "required": ["title",

/tmp/ipython-input-3444937439.py:81: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)


[llm/end] [llm:ChatOpenAI] [3.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"title\": \"Engaging LLM Agents through Structured Argumentation\",\n    \"main_finding\": \"Engaging LLM agents through structured argumentation or discourse facilitates the exchange of reasoning among different agents and guides them toward more accurate answers.\",\n    \"methodology\": \"The study involves engaging large language model (LLM) agents in structured argumentation or discourse to improve their reasoning and accuracy.\",\n    \"key_contributions\": [\n        \"Demonstrates that structured argumentation can lead to more truthful answers and evaluations.\",\n        \"Highlights the potential of enhancing tasks such as machine translation and negotiation through improved reasoning.\",\n        \"Identifies the computational overhead as a significant challenge in implementing these techniques.\"\n    ],\n    \"confidence\": 0.9\n}\n```",
    

In [ ]:
# Cell 17: 案例 2 - 技術問答系統
print("💻 案例 2: 結構化技術問答\n")

class TechnicalQA(BaseModel):
    question_type: str = Field(description="問題類型：概念/方法/比較/實作")
    answer: str = Field(description="詳細回答")
    key_terms: List[str] = Field(description="關鍵術語，2-4 個")
    difficulty: str = Field(description="難度：初級/中級/進階")
    related_topics: List[str] = Field(description="相關主題")
    sources_used: int = Field(description="使用的來源數量")

tech_parser = PydanticOutputParser(pydantic_object=TechnicalQA)

tech_prompt = PromptTemplate(
    template="""你是一個技術專家。根據以下資訊回答問題。

相關資訊:
{context}

技術問題: {question}

{format_instructions}

結構化回答:""",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": tech_parser.get_format_instructions()}
)

query = "multi-agent debate 和傳統的單一 agent 有什麼差異？"

# RAG + Parser 流程
relevant_docs = retriever.get_relevant_documents(query)
context = "\n\n".join([doc.page_content for doc in relevant_docs[:3]])
formatted_prompt = tech_prompt.format(context=context, question=query)
response = stable_chat.invoke([HumanMessage(content=formatted_prompt)])

try:
    qa_result = tech_parser.parse(response.content)

    print("✅ 技術問答結果:\n")
    print(f"❓ 問題類型: {qa_result.question_type}")
    print(f"📊 難度: {qa_result.difficulty}")
    print(f"\n💬 回答:\n{qa_result.answer}")
    print(f"\n🔑 關鍵術語: {', '.join(qa_result.key_terms)}")
    print(f"\n🔗 相關主題:")
    for topic in qa_result.related_topics:
        print(f"  - {topic}")
    print(f"\n📚 使用來源: {qa_result.sources_used} 個片段")

except Exception as e:
    print(f"❌ 解析失敗: {e}")
    print(f"\n原始輸出:\n{response.content}")

In [ ]:
# Cell 18: 案例 3 - 方法比較分析
print("⚖️  案例 3: 研究方法比較分析\n")

class MethodComparison(BaseModel):
    method_name: str = Field(description="方法名稱")
    advantages: List[str] = Field(description="優點，2-3 點")
    disadvantages: List[str] = Field(description="缺點，2-3 點")
    use_cases: List[str] = Field(description="適用場景")
    performance_note: str = Field(description="效能說明")
    recommendation: str = Field(description="使用建議")

comparison_parser = PydanticOutputParser(pydantic_object=MethodComparison)

comparison_prompt = PromptTemplate(
    template="""根據以下資訊，分析研究方法的優缺點。

研究內容:
{context}

分析問題: {question}

{format_instructions}

比較分析:""",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": comparison_parser.get_format_instructions()}
)

query = "分析 multi-agent debate 方法的優缺點"

relevant_docs = retriever.get_relevant_documents(query)
context = "\n\n".join([doc.page_content for doc in relevant_docs[:4]])  # 使用更多片段
formatted_prompt = comparison_prompt.format(context=context, question=query)
response = stable_chat.invoke([HumanMessage(content=formatted_prompt)])

try:
    comparison = comparison_parser.parse(response.content)

    print("✅ 方法比較分析:\n")
    print(f"📌 方法: {comparison.method_name}\n")

    print("✅ 優點:")
    for i, adv in enumerate(comparison.advantages, 1):
        print(f"  {i}. {adv}")

    print("\n❌ 缺點:")
    for i, dis in enumerate(comparison.disadvantages, 1):
        print(f"  {i}. {dis}")

    print("\n🎯 適用場景:")
    for use_case in comparison.use_cases:
        print(f"  • {use_case}")

    print(f"\n📊 效能說明:\n{comparison.performance_note}")
    print(f"\n💡 使用建議:\n{comparison.recommendation}")

except Exception as e:
    print(f"❌ 解析失敗: {e}")
    print(f"\n原始輸出:\n{response.content}")

## Part 6: 進階檢索技術 - BM25、Hybrid Search、Rerank (NEXT WEEK)

In [ ]:
# Cell 15.5: 檢索方法效果比較
print("📊 範例 5: 檢索方法全面比較\n")

import time

def compare_retrieval_methods(query: str):
    """比較不同檢索方法的效果"""
    results = {}

    # 1. Vector 檢索
    start = time.time()
    vector_docs = retriever.invoke(query)[:3]
    vector_time = time.time() - start
    results['Vector'] = {
        'docs': vector_docs,
        'time': vector_time,
        'method': '語義向量檢索'
    }

    # 2. BM25 檢索
    start = time.time()
    bm25_docs = bm25_retriever.invoke(query)[:3]
    bm25_time = time.time() - start
    results['BM25'] = {
        'docs': bm25_docs,
        'time': bm25_time,
        'method': '關鍵字檢索'
    }

    # 3. Hybrid 檢索
    start = time.time()
    hybrid_docs = hybrid_retriever.invoke(query)[:3]
    hybrid_time = time.time() - start
    results['Hybrid'] = {
        'docs': hybrid_docs,
        'time': hybrid_time,
        'method': '混合檢索'
    }

    # 4. Hybrid + Rerank
    start = time.time()
    reranked_docs = hybrid_rerank_retriever(query, k=3, candidate_k=10)
    rerank_time = time.time() - start
    results['Hybrid+Rerank'] = {
        'docs': reranked_docs,
        'time': rerank_time,
        'method': '混合檢索 + 重排序'
    }

    return results

# 測試不同類型的查詢
test_cases = [
    {
        'query': 'multi-agent debate accuracy performance',
        'type': '關鍵字密集型查詢'
    },
    {
        'query': 'How can we improve the reasoning ability of language models?',
        'type': '語義理解型查詢'
    }
]

for case in test_cases:
    query = case['query']
    query_type = case['type']

    print(f"\n{'='*80}")
    print(f"🔬 測試案例: {query_type}")
    print(f"❓ 查詢: {query}\n")

    # 執行比較
    comparison = compare_retrieval_methods(query)

    # 顯示結果
    print("📊 檢索結果比較：\n")

    for method_name, data in comparison.items():
        print(f"{'─'*80}")
        print(f"🔹 方法: {method_name} ({data['method']})")
        print(f"⏱️  耗時: {data['time']*1000:.2f} ms")
        print(f"📄 檢索結果:")

        for i, doc in enumerate(data['docs'], 1):
            preview = doc.page_content[:100].replace('\n', ' ')
            print(f"   {i}. {preview}...")
        print()

print(f"{'='*80}\n")

# 總結表格
print("📈 檢索方法特性總結：\n")
print("┌─────────────────┬──────────────────┬──────────────┬─────────────────┐")
print("│ 方法            │ 優點             │ 缺點         │ 適用場景        │")
print("├─────────────────┼──────────────────┼──────────────┼─────────────────┤")
print("│ Vector          │ 語義理解強       │ 關鍵字弱     │ 語義查詢        │")
print("│ BM25            │ 關鍵字精確       │ 無語義理解   │ 精確匹配        │")
print("│ Hybrid          │ 兼具兩者優點     │ 參數調整複雜 │ 通用查詢        │")
print("│ Hybrid+Rerank   │ 最高準確度       │ 速度較慢     │ 品質要求高      │")
print("└─────────────────┴──────────────────┴──────────────┴─────────────────┘")

print("\n💡 選擇建議：")
print("• 延遲敏感 → Vector 或 BM25")
print("• 品質優先 → Hybrid + Rerank (推薦！)")
print("• 平衡方案 → Hybrid")
print("• 特定領域 → 根據查詢類型選擇")


In [ ]:
# Cell 15.4: 完整的 Hybrid + Rerank RAG Pipeline
print("🚀 範例 4: 完整 Hybrid + Rerank RAG 系統\n")

from langchain_core.runnables import RunnableLambda

print("📘 完整 Pipeline 流程：")
print("1️⃣  Hybrid 檢索：BM25 + Vector 混合檢索")
print("2️⃣  Rerank：使用 Cross-Encoder 重新排序")
print("3️⃣  Context 組合：將最相關文檔組合為上下文")
print("4️⃣  LLM 生成：基於優質上下文生成答案\n")

def hybrid_rerank_retriever(query: str, k: int = 3, candidate_k: int = 10):
    """Hybrid + Rerank 檢索器"""
    # Step 1: Hybrid 檢索
    candidates = hybrid_retriever.invoke(query)[:candidate_k]

    # Step 2: Rerank
    reranked = rerank_documents(query, candidates, top_k=k)

    # 返回文檔（不含分數）
    return [doc for doc, score in reranked]

# 建立完整的 RAG Chain
advanced_rag_prompt = PromptTemplate(
    template="""你是一個專業的學術論文分析助手。請根據以下資訊詳細回答問題。

相關資訊:
{context}

問題: {question}

請提供：
1. 直接回答問題
2. 引用相關證據
3. 如果資訊不足，請說明

回答:""",
    input_variables=["context", "question"]
)

# 測試完整系統
test_queries = [
    "What is the main contribution of this research?",
    "How does multi-agent debate improve performance?",
    "What are the limitations of this approach?"
]

for query in test_queries:
    print(f"\n{'='*70}")
    print(f"❓ 問題: {query}\n")

    # 使用 Hybrid + Rerank 檢索
    print("🔍 執行 Hybrid + Rerank 檢索...")
    relevant_docs = hybrid_rerank_retriever(query, k=3, candidate_k=10)

    print(f"✅ 檢索到 {len(relevant_docs)} 個高相關文檔\n")

    # 組合上下文
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # 生成答案
    print("🤖 生成答案...\n")
    formatted_prompt = advanced_rag_prompt.format(context=context, question=query)
    response = chat_model.invoke([HumanMessage(content=formatted_prompt)])

    print("💬 回答:")
    print(response.content)
    print(f"\n📚 使用來源: {len(relevant_docs)} 個文檔片段")
    print(f"📄 來源頁碼: {', '.join([str(doc.metadata.get('page', 'N/A')) for doc in relevant_docs])}")

print(f"\n{'='*70}\n")
print("✨ 系統優勢總結：")
print("✅ Hybrid Search - 結合關鍵字與語義檢索")
print("✅ Reranker - 精確排序最相關文檔")
print("✅ 高品質上下文 - 提升 LLM 回答準確度")
print("✅ 減少幻覺 - 基於真實文檔內容生成")

In [ ]:
# Cell 15.3: Reranker - 重新排序提升相關性
print("🎯 範例 3: Reranker (重新排序)\n")

from sentence_transformers import CrossEncoder
import numpy as np

print("📘 Reranker 原理：")
print("- 使用 Cross-Encoder 模型對檢索結果重新評分")
print("- Cross-Encoder 同時考慮查詢和文檔，計算相關性分數")
print("- 比 Bi-Encoder (向量檢索) 更精確，但速度較慢")
print("- 適合對初步檢索結果進行精煉\n")

# 載入 Reranker 模型（使用輕量級模型以節省資源）
print("🔄 載入 Reranker 模型...")
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("✅ Reranker 模型載入完成\n")

def rerank_documents(query: str, documents: list, top_k: int = 3):
    """使用 Reranker 對文檔重新排序"""
    # 準備 query-document pairs
    pairs = [[query, doc.page_content] for doc in documents]

    # 計算相關性分數
    scores = reranker.predict(pairs)

    # 按分數排序
    sorted_indices = np.argsort(scores)[::-1][:top_k]

    # 返回排序後的文檔和分數
    reranked_docs = [(documents[i], scores[i]) for i in sorted_indices]
    return reranked_docs

# 測試 Reranker
query = "What are the key findings about multi-agent debate?"

print(f"❓ 查詢: {query}\n")

# 先用 Hybrid 檢索獲取候選文檔
print("🔀 步驟 1: Hybrid 檢索 (獲取 10 個候選)")
candidates = hybrid_retriever.invoke(query)[:10]
print(f"   取得 {len(candidates)} 個候選文檔\n")

# 使用 Reranker 重新排序
print("🎯 步驟 2: Reranker 重新排序")
reranked_results = rerank_documents(query, candidates, top_k=3)

print("\n📊 Reranked 結果 (按相關性排序):\n")
for i, (doc, score) in enumerate(reranked_results, 1):
    print(f"排名 {i} (分數: {score:.4f}):")
    print(f"  內容: {doc.page_content[:120]}...")
    print(f"  來源: 第 {doc.metadata.get('page', 'N/A')} 頁")
    print()

print("💡 Reranker 優勢：")
print("- 更精確的相關性評估")
print("- 減少不相關結果")
print("- 提升 RAG 系統整體品質")

In [ ]:
# Cell 15.2: Hybrid Search - 結合 BM25 與向量檢索
print("🔀 範例 2: Hybrid Search (混合檢索)\n")

from langchain.retrievers import EnsembleRetriever

print("📘 Hybrid Search 原理：")
print("- 結合 BM25 (關鍵字) 和 Vector (語義) 檢索")
print("- 使用加權平均合併兩種檢索結果")
print("- 優點：同時獲得精確匹配和語義理解\n")

# 建立 Ensemble Retriever (混合檢索器)
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],  # BM25 + Vector
    weights=[0.5, 0.5]  # 各佔 50% 權重
)

# 比較測試：使用更語義化的查詢
test_queries = [
    "What is the main contribution of this paper?",  # 語義查詢
    "multi-agent debate performance results",  # 關鍵字查詢
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"❓ 查詢: {query}\n")

    # Vector 檢索
    print("🎯 Vector 檢索 (前 2 個):")
    vector_results = retriever.invoke(query)[:2]
    for i, doc in enumerate(vector_results, 1):
        print(f"  {i}. {doc.page_content[:80]}... (頁 {doc.metadata.get('page', 'N/A')})")

    # BM25 檢索
    print("\n🔍 BM25 檢索 (前 2 個):")
    bm25_results = bm25_retriever.invoke(query)[:2]
    for i, doc in enumerate(bm25_results, 1):
        print(f"  {i}. {doc.page_content[:80]}... (頁 {doc.metadata.get('page', 'N/A')})")

    # Hybrid 檢索
    print("\n🔀 Hybrid 檢索 (前 3 個):")
    hybrid_results = hybrid_retriever.invoke(query)[:3]
    for i, doc in enumerate(hybrid_results, 1):
        print(f"  {i}. {doc.page_content[:80]}... (頁 {doc.metadata.get('page', 'N/A')})")

print(f"\n{'='*60}")
print("\n💡 觀察重點：")
print("- 語義查詢：Vector 表現更好，理解問題意圖")
print("- 關鍵字查詢：BM25 表現更好，精確匹配")
print("- Hybrid 結合兩者優勢，更穩健")

In [ ]:
# Cell 15.1: BM25 Retriever - 關鍵字檢索
print("🔍 範例 1: BM25 Retriever (關鍵字檢索)\n")

from langchain.retrievers import BM25Retriever

print("📘 BM25 演算法說明：")
print("- BM25 (Best Matching 25) 是一種基於 TF-IDF 的排序函數")
print("- 優點：關鍵字匹配精確、不需要向量化、速度快")
print("- 缺點：無法理解語義、對同義詞不敏感\n")

# 建立 BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 3  # 返回前 3 個結果

# 測試查詢
query = "multi-agent debate methodology"

print(f"❓ 查詢: {query}\n")
print("🔎 BM25 檢索結果:\n")

bm25_results = bm25_retriever.invoke(query)

for i, doc in enumerate(bm25_results, 1):
    print(f"📄 結果 {i}:")
    print(f"內容: {doc.page_content[:150]}...")
    print(f"來源: 第 {doc.metadata.get('page', 'N/A')} 頁")
    print()